In [11]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
import json
import pprint

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from config import dbuser, dbpasswd, dburi, dbport, dbname

### Store CSV into DataFrame

In [12]:
csv_file = "KCPD_Crime_Data_2018_cleaned.csv"
kcmo_data_df = pd.read_csv(csv_file)
kcmo_data_df.head()

,#,Report_No,Reported_Date,Reported_Time,Offense,Description,Address,City,Zip_Code,Area,...,Involvement,Race,Sex,Age,Firearm_Used_Flag,Location,Lat_Lng,Lat,Lng,18325
0,2,180097829,12/20/2018,15:52,401,Aggravated Assault,500 BOOTH AV,KANSAS CITY,64124,EPD,...,SUS,W,M,18,N,"500 BOOTH AV\nKANSAS CITY 64124\n(39.105511, -...","39.105511, -94.482144",39.105511,-94.482144,NaN
1,4,180097829,12/20/2018,15:52,401,Aggravated Assault,500 BOOTH AV,KANSAS CITY,64124,EPD,...,SUS,P,M,18,N,"500 BOOTH AV\nKANSAS CITY 64124\n(39.105511, -...","39.105511, -94.482144",39.105511,-94.482144,NaN
2,9,180093115,12/3/2018,17:07,401,Aggravated Assault,4100 FLORA AV,KANSAS CITY,64110,CPD,...,SUS,W,F,18,Y,"4100 FLORA AV\nKANSAS CITY 64110\n(39.052161, ...","39.052161, -94.566533",39.052161,-94.566533,NaN
3,12,180091999,11/29/2018,14:11,401,Aggravated Assault,900 E 9 ST,KANSAS CITY,64108,CPD,...,SUS,W,M,18,N,"900 E 9 ST\nKANSAS CITY 64108\n(39.103158, -94...","39.103158, -94.57202",39.103150,-94.57202,NaN
4,17,180089325,11/18/2018,12:28,401,Aggravated Assault,6000 E 10 ST,KANSAS CITY,64126,EPD,...,SUS,U,M,18,Y,"6000 E 10 ST\nKANSAS CITY 64126\n(39.100277, -...","39.100277, -94.511176",39.100277,-94.511176,NaN


### Create new data with select columns

In [13]:
new_kcmo_data_df = kcmo_data_df[['Report_No', 'Reported_Date', 'Reported_Time', 'Offense', \
                                 'Description', 'Address', 'City', 'Zip_Code', 'Area', \
                                 'Invl_No', 'Involvement', 'Race', 'Sex', 'Age', \
                                 'Firearm_Used_Flag', 'Location', 'Lat_Lng', 'Lat', 'Lng']].copy()
new_kcmo_data_df.head()

,Report_No,Reported_Date,Reported_Time,Offense,Description,Address,City,Zip_Code,Area,Invl_No,Involvement,Race,Sex,Age,Firearm_Used_Flag,Location,Lat_Lng,Lat,Lng
0,180097829,12/20/2018,15:52,401,Aggravated Assault,500 BOOTH AV,KANSAS CITY,64124,EPD,2,SUS,W,M,18,N,"500 BOOTH AV\nKANSAS CITY 64124\n(39.105511, -...","39.105511, -94.482144",39.105511,-94.482144
1,180097829,12/20/2018,15:52,401,Aggravated Assault,500 BOOTH AV,KANSAS CITY,64124,EPD,1,SUS,P,M,18,N,"500 BOOTH AV\nKANSAS CITY 64124\n(39.105511, -...","39.105511, -94.482144",39.105511,-94.482144
2,180093115,12/3/2018,17:07,401,Aggravated Assault,4100 FLORA AV,KANSAS CITY,64110,CPD,1,SUS,W,F,18,Y,"4100 FLORA AV\nKANSAS CITY 64110\n(39.052161, ...","39.052161, -94.566533",39.052161,-94.566533
3,180091999,11/29/2018,14:11,401,Aggravated Assault,900 E 9 ST,KANSAS CITY,64108,CPD,2,SUS,W,M,18,N,"900 E 9 ST\nKANSAS CITY 64108\n(39.103158, -94...","39.103158, -94.57202",39.103150,-94.57202
4,180089325,11/18/2018,12:28,401,Aggravated Assault,6000 E 10 ST,KANSAS CITY,64126,EPD,1,SUS,U,M,18,Y,"6000 E 10 ST\nKANSAS CITY 64126\n(39.100277, -...","39.100277, -94.511176",39.100277,-94.511176


### Connect to local database

In [14]:
engine = create_engine(f"mysql://{dbuser}:{dbpasswd}@{dburi}:{dbport}/{dbname}")
Base.metadata.create_all(engine)

### Check for tables

In [15]:
engine.table_names()

[]

### Use pandas to load csv converted DataFrame into database

In [16]:
new_kcmo_data_df.to_sql(name='crime_data_2018', con=engine, if_exists='replace', index=True)

In [7]:
pd.read_sql_query('select * from crime_data_2018', con=engine).head()

,index,Report_No,Reported_Date,Reported_Time,Offense,Description,Address,City,Zip_Code,Area,Invl_No,Involvement,Race,Sex,Age,Firearm_Used_Flag,Location,Lat_Lng,Lat,Lng
0,0,180097829,12/20/2018,15:52,401,Aggravated Assault,500 BOOTH AV,KANSAS CITY,64124,EPD,2,SUS,W,M,18,N,"500 BOOTH AV\nKANSAS CITY 64124\n(39.105511, -...","39.105511, -94.482144",39.105511,-94.482144
1,1,180097829,12/20/2018,15:52,401,Aggravated Assault,500 BOOTH AV,KANSAS CITY,64124,EPD,1,SUS,P,M,18,N,"500 BOOTH AV\nKANSAS CITY 64124\n(39.105511, -...","39.105511, -94.482144",39.105511,-94.482144
2,2,180093115,12/3/2018,17:07,401,Aggravated Assault,4100 FLORA AV,KANSAS CITY,64110,CPD,1,SUS,W,F,18,Y,"4100 FLORA AV\nKANSAS CITY 64110\n(39.052161, ...","39.052161, -94.566533",39.052161,-94.566533
3,3,180091999,11/29/2018,14:11,401,Aggravated Assault,900 E 9 ST,KANSAS CITY,64108,CPD,2,SUS,W,M,18,N,"900 E 9 ST\nKANSAS CITY 64108\n(39.103158, -94...","39.103158, -94.57202",39.103150,-94.57202
4,4,180089325,11/18/2018,12:28,401,Aggravated Assault,6000 E 10 ST,KANSAS CITY,64126,EPD,1,SUS,U,M,18,Y,"6000 E 10 ST\nKANSAS CITY 64126\n(39.100277, -...","39.100277, -94.511176",39.100277,-94.511176


### Confirm data and write full dataset to a file in JSON format

In [8]:
temp_json = new_kcmo_data_df.head(3).to_json(orient='records')
new_kcmo_data_df.to_json(orient='records', path_or_buf='kcpd_crime.json')

### Write the temp data to confirm it worked

In [9]:
parsed = json.loads(temp_json)
print(json.dumps(parsed, indent=4, sort_keys=True))
# pprint(temp_json)



[
    {
        "Address": "500  BOOTH AV",
        "Age": 18,
        "Area": "EPD",
        "City": "KANSAS CITY",
        "Description": "Aggravated Assault",
        "Firearm_Used_Flag": "N",
        "Invl_No": 2,
        "Involvement": "SUS",
        "Lat": 39.105511,
        "Lat_Lng": "39.105511, -94.482144",
        "Lng": "-94.482144",
        "Location": "500 BOOTH AV\nKANSAS CITY 64124\n(39.105511, -94.482144)",
        "Offense": 401,
        "Race": "W",
        "Report_No": 180097829,
        "Reported_Date": "12/20/2018",
        "Reported_Time": "15:52",
        "Sex": "M",
        "Zip_Code": 64124
    },
    {
        "Address": "500  BOOTH AV",
        "Age": 18,
        "Area": "EPD",
        "City": "KANSAS CITY",
        "Description": "Aggravated Assault",
        "Firearm_Used_Flag": "N",
        "Invl_No": 1,
        "Involvement": "SUS",
        "Lat": 39.105511,
        "Lat_Lng": "39.105511, -94.482144",
        "Lng": "-94.482144",
        "Location": "500 

## Create Description Summary Info

In [10]:
new_kcmo_data_df.columns

Index(['Report_No', 'Reported_Date', 'Reported_Time', 'Offense', 'Description',
       'Address', 'City', 'Zip_Code', 'Area', 'Invl_No', 'Involvement', 'Race',
       'Sex', 'Age', 'Firearm_Used_Flag', 'Location', 'Lat_Lng', 'Lat', 'Lng'],
      dtype='object')

In [11]:
# Using GroupBy in order to separate the data into fields according to "Description"
grouped_desc_df = new_kcmo_data_df.groupby(['Description'])

# In order to be visualized, a data function must be used...
desc_counts = grouped_desc_df['Description'].count()
# desc_counts.set_index('Description', inplace=True)
desc_counts.head()


Description
Aggravated Assault    1725
Animal Cruelty           2
Armed Robbery          317
Arson                   33
Attempt Suicide          6
Name: Description, dtype: int64

In [12]:
total_crimes = new_kcmo_data_df['Description'].count()
total_crimes

18324

In [13]:
# Creating a new DataFrame using both duration, count and the calculated percent
desc_count_df = pd.DataFrame({ "Description": desc_counts.index, 
                              "Count": desc_counts, 
                              "Percent": desc_counts / total_crimes * 100})
desc_count_df.head()


,Description,Count,Percent
Description,,,
Aggravated Assault,Aggravated Assault,1725,9.413883
Animal Cruelty,Animal Cruelty,2,0.010915
Armed Robbery,Armed Robbery,317,1.729972
Arson,Arson,33,0.180092
Attempt Suicide,Attempt Suicide,6,0.032744


### Use pandas to load DataFrame into database

In [14]:
desc_count_df.columns

Index(['Description', 'Count', 'Percent'], dtype='object')

In [15]:
# Create MySQL table and write Desc_count DF to this MySQL table
desc_count_df.to_sql(name='desc_summary', con=engine, if_exists='replace', index=False)
desc_count_df.head()

,Description,Count,Percent
Description,,,
Aggravated Assault,Aggravated Assault,1725,9.413883
Animal Cruelty,Animal Cruelty,2,0.010915
Armed Robbery,Armed Robbery,317,1.729972
Arson,Arson,33,0.180092
Attempt Suicide,Attempt Suicide,6,0.032744


### Write the temp data to confirm it worked

In [16]:
temp_desc_json = desc_count_df.head(3).to_json(orient='records')
desc_count_df.to_json(orient='records', path_or_buf='desc_summary.json')

In [17]:
parsed = json.loads(temp_desc_json)
print(json.dumps(parsed, indent=4, sort_keys=True))


[
    {
        "Count": 1725,
        "Description": "Aggravated Assault",
        "Percent": 9.4138834316
    },
    {
        "Count": 2,
        "Description": "Animal Cruelty",
        "Percent": 0.0109146475
    },
    {
        "Count": 317,
        "Description": "Armed Robbery",
        "Percent": 1.7299716219
    }
]


## Create Age Summary Info

In [18]:
# Using GroupBy in order to separate the data into fields according to "Description"
grouped_sex_df = new_kcmo_data_df.groupby(['Sex'])

# In order to be visualized, a data function must be used...
sex_counts = grouped_sex_df['Sex'].count()
# desc_counts.set_index('Description', inplace=True)
sex_counts.head()


Sex
F     5360
M    12874
U       90
Name: Sex, dtype: int64

In [19]:
# NOTE:
# total_crimes = new_kcmo_data_df['Description'].count()
# total_crimes = 18324

In [20]:
# Creating a new DataFrame using both duration, count and the calculated percent
sex_count_df = pd.DataFrame({ "Sex": sex_counts.index, 
                              "Count": sex_counts, 
                              "Percent": sex_counts / total_crimes * 100})
sex_count_df.head()


,Sex,Count,Percent
Sex,,,
F,F,5360,29.251255
M,M,12874,70.257586
U,U,90,0.491159


### Use pandas to load DataFrame into database

In [21]:
sex_count_df.columns

Index(['Sex', 'Count', 'Percent'], dtype='object')

In [22]:
# Create MySQL table and write Desc_count DF to this MySQL table
sex_count_df.to_sql(name='sex_summary', con=engine, if_exists='replace', index=False)
sex_count_df.head()

,Sex,Count,Percent
Sex,,,
F,F,5360,29.251255
M,M,12874,70.257586
U,U,90,0.491159


### Write the temp data to confirm it worked

In [23]:
temp_sex_json = sex_count_df.head(3).to_json(orient='records')
sex_count_df.to_json(orient='records', path_or_buf='sex_summary.json')

In [24]:
parsed = json.loads(temp_sex_json)
print(json.dumps(parsed, indent=4, sort_keys=True))


[
    {
        "Count": 5360,
        "Percent": 29.2512551845,
        "Sex": "F"
    },
    {
        "Count": 12874,
        "Percent": 70.25758568,
        "Sex": "M"
    },
    {
        "Count": 90,
        "Percent": 0.4911591356,
        "Sex": "U"
    }
]
